In [1]:
import pandas as pd
import numpy as np
import functools as ft
import itertools as it
import multiprocessing as mp
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [163]:
df = pd.read_csv('../data/go_synonym_data.txt', header=None, sep='\t', low_memory=False,)
df.columns = ['ID', 'Gene/Product', 'Name', 'GO Class Labels', 'Synonyms']
print(df.shape)
df.head(3)

(19712, 5)


,ID,Gene/Product,Name,GO Class Labels,Synonyms
0,UniProtKB:P25788,PSMA3,Proteasome subunit alpha type-3,protein deubiquitination|endopeptidase activit...,HC8|PSC8
1,UniProtKB:P25787,PSMA2,Proteasome subunit alpha type-2,protein deubiquitination|endopeptidase activit...,HC3|PSC3
2,UniProtKB:P25786,PSMA1,Proteasome subunit alpha type-1,protein deubiquitination|endopeptidase activit...,HC2|NU|PROS30|PSC2


Below is an example of what the labels for the BOW are

In [18]:
'''
t = pd.read_csv('../reproduced/consensusclusters_spinglass_greaterthan90percent.txt', sep = '\t')
out = []
for i in t['cluster'].unique():
    out.append(t[t['cluster']==i]['names'].tolist())'''

In [27]:
'''with open('clusters_for_mathematica.txt', 'a') as f:
    for i in out:
        f.write(str(i).replace('[', '').replace(']','').replace(',', '').replace('\'', '') + '\n')'''

In [28]:
print(len(df['ID'].unique()))

19712


In [29]:
print(len(df['Gene/Product'].unique()))

19659


In [173]:
def splitter(x):
    try:
        temp = x.split('|')
    except:
        temp = []
    return temp
        

In [174]:
df['Synonyms'] = df['Synonyms'].apply(splitter)

In [44]:
kinase_network_df = pd.read_csv('../data/Full_Kinome_Network_Compiled_no_header.txt', header=None, sep='\t')
known_kinases = list(set(kinase_network_df[0]) | set(kinase_network_df[1]))
print(len(known_kinases))

473


In [45]:
df[df['Gene/Product']=='RAF1']

,ID,Gene/Product,Name,GO Class Labels,Synonyms
18381,UniProtKB:P04049,RAF1,RAF proto-oncogene serine/threonine-protein ki...,nuclear speck|Golgi apparatus|face development...,RAF


In [46]:
df.dtypes

ID                 object
Gene/Product       object
Name               object
GO Class Labels    object
Synonyms           object
dtype: object

In [47]:
print(len(list(filter(lambda x: x != 'nan', it.chain.from_iterable(df['Synonyms'].apply(lambda x: str(x).split('|')))))))
print(len(set(filter(lambda x: x != 'nan', it.chain.from_iterable(df['Synonyms'].apply(lambda x: str(x).split('|')))))))

23978
23297


In [48]:
set_of_synonyms_present = set(filter(lambda x: x != 'nan', it.chain.from_iterable(df['Synonyms'].apply(lambda x: str(x).split('|')))))
set_of_gene_products = set(df['Gene/Product'].values)

In [49]:
missing = []

for k in known_kinases:
    if k not in set_of_synonyms_present and k not in set_of_gene_products:
        missing.append(k)

In [50]:
len(missing)

155

In [42]:
'ACV1C' in set_of_gene_products or 'ACV1C' in set_of_synonyms_present

False

In [41]:
'ACVR1C' in set_of_gene_products # the ACV1C protein is not a synonym

True

In [88]:
alias = pd.read_excel('../data/KINASESmasterlist_w_Aliases.xlsx', header = 0)

In [101]:
kin_map = alias.set_index('Uniprot Protein')['MS Gene'].to_dict()

In [115]:
go_aliases = set([x for y in ser for x in y if 'UniProt' not in x and 'PTN' not in x])

In [125]:
all_aliases = alias.set_index('Uniprot Protein')['Aliases (Conservative)'].dropna().apply(lambda x: x.split(',')).to_dict()

In [193]:
go_dat = {}

for k in known_kinases:
    temp = df[df['Gene/Product']==k]
    if(temp.shape[0] == 0):
        temp = df[df['Synonyms'].apply(lambda x: k in x)]
        if(temp.shape[0] == 0):
            temp = df[df['Gene/Product']==kin_map[k]]
            if(temp.shape[0] == 0):
                temp = df[df['Synonyms'].apply(lambda x: kin_map[k] in x)]
                if(temp.shape[0]==0):
                    r = all_aliases.get(k)
                    if(r is not None):
                        for a in r:
                            temp = df[df['Gene/Product']==a]
                            if(temp.shape[0] ==0):
                                temp = df[df['Synonyms'].apply(lambda x: a in x)]
                                if(temp.shape[0]==0):
                                    pass
                                else:
                                    go_dat[k] = temp
                                    break
                            else:
                                go_dat[k] = temp
                else:
                    go_dat[k] = temp
            else:
                go_dat[k] = temp
        else:
            go_dat[k] = temp
    else:
        go_dat[k] = temp

In [194]:
go_dat[list(go_dat.keys())[0]]

,ID,Gene/Product,Name,GO Class Labels,Synonyms
17878,UniProtKB:P54762,EPHB1,Ephrin type-B receptor 1,peptidyl-tyrosine phosphorylation|axon guidanc...,"[ELK, EPHT2, HEK6, NET]"


In [195]:
for x in go_dat.keys():
    try:
        go_dat[x]['GO Class Labels'].values[0]
    except:
        print(x)

In [196]:
go_dat[list(go_dat.keys())[0]]['GO Class Labels'].values[0]

'peptidyl-tyrosine phosphorylation|axon guidance|central nervous system projection neuron axonogenesis|protein autophosphorylation|neurogenesis|optic nerve morphogenesis|protein-containing complex binding|cell chemotaxis|dendrite|cytosol|glutamatergic synapse|positive regulation of synapse assembly|axon|receptor complex|negative regulation of satellite cell differentiation|plasma membrane|dendritic spine development|negative regulation of skeletal muscle satellite cell proliferation|transmembrane-ephrin receptor activity|dendritic spine morphogenesis|early endosome membrane|protein binding|ephrin receptor signaling pathway|establishment of cell polarity|filopodium tip|modulation of chemical synaptic transmission|cell-substrate adhesion|detection of temperature stimulus involved in sensory perception of pain|neural precursor cell proliferation|axon guidance receptor activity|extracellular region|neuron projection|immunological synapse formation|ATP binding|integral component of plasma m

In [197]:
# find any kinases where multiple gene/product IDs were returned
for x in go_dat.keys():
    if(go_dat[x].shape[0] > 1):
        print(x)

P4K2B


In [208]:
go_dat['P4K2B'].groupby('Gene/Product').apply(sum)['GO Class Labels'][0]

'phosphatidylinositol biosynthetic process|endosome|phosphorylation|1-phosphatidylinositol 4-kinase activity|phosphatidylinositol phosphorylation|kinase activity|plasma membrane|Golgi organization|endosome organization|membrane|cytosol|ATP binding|trans-Golgi networktrans-Golgi network|endosome|endosome organization|Golgi organization|phosphatidylinositol phosphorylation|1-phosphatidylinositol 4-kinase activity|phosphorylation|kinase activity|plasma membrane'

In [217]:
go_dat['P4K2B'].iloc[0]['GO Class Labels']

'phosphatidylinositol biosynthetic process|endosome|phosphorylation|1-phosphatidylinositol 4-kinase activity|phosphatidylinositol phosphorylation|kinase activity|plasma membrane|Golgi organization|endosome organization|membrane|cytosol|ATP binding|trans-Golgi network'

In [219]:
temp = go_dat['P4K2B'].iloc[0]
temp['GO Class Labels']='|'.join(list(set(go_dat['P4K2B']['GO Class Labels'].iloc[0].split('|')) | set(go_dat['P4K2B']['GO Class Labels'].iloc[1].split('|'))))
go_dat['P4K2B'] = temp
#print(np.array(temp['GO Class Labels']))#.values)

In [228]:
def helper(x):
    try:
        temp = go_dat[x]['GO Class Labels'].values[0].split('|')
    except:
        temp = go_dat[x]['GO Class Labels'].split('|')
    return temp

In [229]:
just_labels = {x:helper(x) for x in go_dat.keys()}

In [231]:
agg_labels = [x for y in labeled_kinases.values for x in y]

In [232]:
from collections import Counter

c = Counter(agg_labels)

In [233]:
c.most_common(n=50)

[('ATP binding', 460),
 ('protein binding', 402),
 ('cytoplasm', 306),
 ('protein phosphorylation', 299),
 ('protein serine/threonine kinase activity', 290),
 ('cytosol', 278),
 ('nucleus', 229),
 ('plasma membrane', 177),
 ('protein kinase activity', 156),
 ('protein autophosphorylation', 148),
 ('nucleoplasm', 140),
 ('intracellular signal transduction', 133),
 ('peptidyl-serine phosphorylation', 113),
 ('signal transduction', 108),
 ('peptidyl-tyrosine phosphorylation', 97),
 ('metal ion binding', 97),
 ('negative regulation of apoptotic process', 95),
 ('protein tyrosine kinase activity', 80),
 ('cell differentiation', 80),
 ('identical protein binding', 73),
 ('membrane', 72),
 ('MAPK cascade', 70),
 ('receptor complex', 68),
 ('integral component of plasma membrane', 68),
 ('positive regulation of ERK1 and ERK2 cascade', 65),
 ('magnesium ion binding', 61),
 ('kinase activity', 58),
 ('extracellular exosome', 57),
 ('apoptotic process', 57),
 ('negative regulation of signal trans

In [234]:
stopwords = ['ATP binding','protein binding', 'protein phosphorylation','protein serine/threonine kinase activity']

In [235]:
def stophelper(x):
    try:
        temp = go_dat[x]['GO Class Labels'].values[0].split('|')
    except:
        temp = go_dat[x]['GO Class Labels'].split('|')
    return list(filter(lambda x: x not in stopwords, temp))

In [236]:
kinase_labels = {x:stophelper(x) for x in go_dat.keys()}
labeled_kinases = pd.Series(kinase_labels)

In [238]:
labeled_kinases.to_csv('kinase_go_annotations.csv')